In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# workaround via specifying an invalid value first
# %config Application.log_level='WORKAROUND'
# => fails, necessary on Fedora 27, ipython3 6.2.1
%config Application.log_level='INFO'

In [3]:
import logging
logging.getLogger().setLevel(logging.INFO)
log = logging.getLogger()
log.debug('Test debug')

In [4]:
from datetime import datetime
import pickle

import pandas as pd
import tqdm

In [5]:
from censorability_monitor.data import process_transaction, get_addresses_from_receipt

In [1]:
from web3 import Web3, IPCProvider

geth_ipc = '/media/Warehouse/Warehouse/Ethereum/data/.ethereum/geth.ipc'
w3 = Web3(IPCProvider(geth_ipc))
# Check
print ("Latest Ethereum block number" , w3.eth.blockNumber)

Latest Ethereum block number 16490481


In [7]:
contracts_df = pd.read_csv('contracts_221220_221231.csv')
contracts_df.head()

,contract,transactions
0,0xdAC17F958D2ee523a2206206994597C13D831ec7,1042798
1,0x00000000006c3852cbEf3e08E8dF289169EdE581,564820
2,0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45,467017
3,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,273982
4,0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D,224322


In [8]:
with open('contracts_db.pickle', 'rb') as f:
    contract_db = pickle.load(f)

In [9]:
data = pd.read_csv('data/transactions_from_to_hash.csv')

In [10]:
len(data)

11527423

In [17]:
addresses_from_receipts = {}
n_without_from = 0
n_without_to = 0
not_contract_transaction = 0

for i, row in tqdm.tqdm(data.iterrows(), total=len(data)):
    if row['is_contract_transaction'] is False:
        not_contract_transaction += 1
        continue
    contract_address = row['to']
    # tx = w3.eth.get_transaction(row['tx_hash'])
    tx_receipt = w3.eth.get_transaction_receipt(row['tx_hash'])
    # addresses = process_transaction(row['tx_hash'], tx, tx_receipt, contract_address, w3)
    addresses = get_addresses_from_receipt(tx_receipt)
    if row['to'] not in addresses:
        addresses.add(row['to'].lower())
        n_without_to += 1
    if row['from'] not in addresses:
        addresses.add(row['from'].lower())
        n_without_from += 1
    addresses_from_receipts[row['tx_hash']] = list(addresses)
    if i % 100000 == 0:
        with open('addresses_from_receipts.pickle', 'wb') as f:
            pickle.dump(addresses_from_receipts, f)
with open('addresses_from_receipts.pickle', 'wb') as f:
    pickle.dump(addresses_from_receipts, f)

 88%|████████▊ | 10106515/11527423 [5:12:04<39:23, 601.12it/s]  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 11527423/11527423 [5:56:07<00:00, 539.47it/s]  


In [18]:
len(addresses_from_receipts)

8357981

In [22]:
blocks = {}
for i in tqdm.tqdm(range(16221291, 16307292 + 1)):
    block_data = w3.eth.get_block(i)
    blocks[i] = block_data

with open('data/twelve_days_blocks.pickle', 'wb') as f:
    pickle.dump(blocks, f)

100%|██████████| 86002/86002 [03:56<00:00, 362.96it/s]


In [65]:
addresses

{'0x0000000000000000000000000000000000000000',
 '0x0000000000000000000000000000000000002467',
 '0x498b859d2e59958e209d7dd262324c8d31b12b12',
 '0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab',
 '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5'}

In [13]:
print(contract_address)
contract_db[contract_address]['abi']

0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5


[{'inputs': [{'internalType': 'address',
    'name': '_contractOwner',
    'type': 'address'}],
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'stateMutability': 'payable', 'type': 'fallback'},
 {'stateMutability': 'payable', 'type': 'receive'}]

In [14]:
for el in contract_db[contract_address]['abi']:
    el_type = el.get('type', None)
    if el_type:
        print(el_type)

constructor
fallback
receive


In [16]:
contract = w3.eth.contract(address=contract_address, abi=contract_db[contract_address]['abi'])

In [17]:
contract.functions.facetAddresses().call()

ABIFunctionNotFound: ("The function 'facetAddresses' was not found in this contract's abi. ", 'Are you sure you provided the correct contract abi?')

In [11]:
tx

AttributeDict({'blockHash': HexBytes('0x47113ab19d73a5c56854ae24273a1caf43b477a175964557464f92078f8317d0'),
 'blockNumber': 16221291,
 'from': '0x498B859D2e59958E209d7DD262324C8D31b12B12',
 'gas': 242494,
 'gasPrice': 117114004578,
 'maxFeePerGas': 117114004578,
 'maxPriorityFeePerGas': 117114004578,
 'hash': HexBytes('0x14e970dba71ef93b42f337d3494c4d6e8fee4d83ce388cd911076d3163f4ed81'),
 'input': '0xfc06d2a6',
 'nonce': 12269,
 'to': '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5',
 'transactionIndex': 0,
 'value': 0,
 'type': '0x2',
 'accessList': [],
 'chainId': '0x1',
 'v': 1,
 'r': HexBytes('0xae34479899815d7009d024f639c0430c8690e44006a85eadd1c98a06fa6d7f6c'),
 's': HexBytes('0x64b730cc097b328a0216e1113b110c24b2fbd6c0290ec4cd5731d360f3096b67')})

In [66]:
tx_receipt

AttributeDict({'blockHash': HexBytes('0x47113ab19d73a5c56854ae24273a1caf43b477a175964557464f92078f8317d0'),
 'blockNumber': 16221291,
 'contractAddress': None,
 'cumulativeGasUsed': 189564,
 'effectiveGasPrice': 117114004578,
 'from': '0x498B859D2e59958E209d7DD262324C8D31b12B12',
 'gasUsed': 189564,
 'logs': [AttributeDict({'address': '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5',
   'topics': [HexBytes('0xb360bcf4b60112f485fd94b599df45181250ef0e80538be7b334728ab0990b1a'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000002467')],
   'data': '0x',
   'blockNumber': 16221291,
   'transactionHash': HexBytes('0x14e970dba71ef93b42f337d3494c4d6e8fee4d83ce388cd911076d3163f4ed81'),
   'transactionIndex': 0,
   'blockHash': HexBytes('0x47113ab19d73a5c56854ae24273a1caf43b477a175964557464f92078f8317d0'),
   'logIndex': 0,
   'removed': False}),
  AttributeDict({'address': '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5',
   'topics': [HexBytes('0x0e0c101fa6afb12838450cfd752d

In [63]:
receipt_possible_addresses = set()

for log in tx_receipt['logs']:
    if 'address' in log:
        receipt_possible_addresses.add(log['address'])
    for el in log['topics']:
        len_el = len(el)
        if len_el == 20:
            receipt_possible_addresses.add(el)
        elif len_el > 20:
            prefix = el[:len_el - 20]
            if len(prefix) == prefix.count(b'\x00'):
                receipt_possible_addresses.add(el[len_el - 20:].hex())
receipt_possible_addresses

{'0x0000000000000000000000000000000000000000',
 '0x0000000000000000000000000000000000002467',
 '0x498b859d2e59958e209d7dd262324c8d31b12b12',
 '0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab',
 '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5'}

In [55]:
hb = tx_receipt['logs'][0]['topics'][1]

prefix = hb[:len(hb) - 20]
if 

12

In [58]:
bytes(prefix)

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [2]:
pending = w3.geth.txpool.content()

In [7]:
len(pending['pending'])

3641

In [8]:
len(pending['queued'])

378

In [12]:
for k, v in pending['pending'].items():
    print(k)
    print(v)
    break

0x000000aACE0e0959C134069caA419F0F23050eA0
AttributeDict({'329': AttributeDict({'blockHash': None, 'blockNumber': None, 'from': '0x000000aace0e0959c134069caa419f0f23050ea0', 'gas': '0x7a120', 'gasPrice': '0x2cb417800', 'hash': '0x80fbc406e1e201cecd93fbdca770bb8b31158b902cc478a4c14db64982389373', 'input': '0xf4dd8c36', 'nonce': '0x149', 'to': '0x43c41ac895f22f068f6ec875822b13421a9086cc', 'transactionIndex': None, 'value': '0x0', 'type': '0x0', 'chainId': '0x1', 'v': '0x26', 'r': '0xbc4d7e2849b262865e4efa9059f9fd35088757995291fcc7ed4c81e05fb2ce7d', 's': '0x3b423f7696a2a9b6eeaa1dd11a9641912e0d53e7ae1c37d0b0f687d2fef09438'})})
